In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize 
import sys
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [2]:
### RUN ONLY ONCE ###

# business_json_path = 'yelp_academic_dataset_business.json'
# df_b = pd.read_json(business_json_path, lines=True)
# # 1 = open, 0 = closed
# df_b = df_b[df_b['is_open']==1]
# drop_columns = ['hours','is_open','review_count','address']
# df_b = df_b.drop(drop_columns, axis=1)
# df_b.to_csv('bussines.csv',index=False)

In [3]:
### RUN ONLY ONCE ###


# review_json_path = 'yelp_academic_dataset_review.json'
# size = 1000000
# review = pd.read_json(review_json_path, lines=True,
#                       dtype={'review_id':str,'user_id':str,
#                              'business_id':str,'stars':int,
#                              'date':str,'text':str,'useful':int,
#                              'funny':int,'cool':int},
#                       chunksize=size)

In [4]:
### RUN ONLY ONCE ###


# There are multiple chunks to be read
# chunk_list = []
# for chunk_review in review:
#     # Drop columns that aren't needed
#     chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
#     # Renaming column name to avoid conflict with business overall star rating
#     chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
#     # Inner merge with edited business file so only reviews related to the business remain
#     chunk_merged = pd.merge(df_b, chunk_review, on='business_id', how='inner')
#     # Show feedback on progress
#     print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
#     chunk_list.append(chunk_merged)
# # # After trimming down the review file, concatenate all relevant data back to one dataframe
# df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
#df.to_csv('reviews.csv',index=False)

In [ ]:
### KEEP ONLY RESTARUNTS ###
business_R = df_b[df_b['categories'].str.contains('Restaurants',case=False, na=False)]

In [5]:
### Read bussines.csv ###
### using df_b.loc[id of bussines] to find it ##

df_b = pd.read_csv('bussines.csv',dtype={'name':str,
                              'city':str, 'state':str, 'postal_code':str,
                              'latitude':float, 'longitude':float,             
                              'business_id':str,'stars':float,
                               'attributes':str,'categories':str},index_col='business_id')
#df_b.head()
df_explode = df_b.assign(categories = df_b.categories.str.split(', ')).explode('categories')
df_explode.categories.value_counts()
#df_b.head()

In [6]:
### Read reviews.csv with business id ###
### This csv contains only reviews for open bussines, as they were filtered at the begining ###
### using df.loc[id of user] to find all of his reviews ##


df_r = pd.read_csv("reviews_cleared.csv",dtype={'user_id':str,            
                              'business_id':str,'review_stars':np.int8,'text':str},parse_dates=["date"],index_col='user_id')

In [7]:
### Read users.csv ###
### This csv contains only reviews for open bussines, as they were filtered at the begining ###
### using df.loc[id of user] to find all of his friends ##
df_u = pd.read_csv("users.csv",dtype={'friends':str},index_col='user_id')

In [8]:
### Read chekin.csv ###
### This csv contains only chekins for open bussines, as they were filtered at the begining ###
### using df.loc[id of bussines] to find all of his friends ##
df_c = pd.read_csv("checkin.csv",parse_dates=["dates"],index_col='business_id')

In [9]:
### Read tips.csv ###
### This csv contains only chekins for open bussines, as they were filtered at the begining ###
### using df.loc[id of bussines] to find all of his friends ##
df_t = pd.read_csv("tips.csv",dtype={'user_id':str,'text':str}, index_col='business_id',parse_dates=["date"])

In [10]:
df_b.head()

,name,city,state,postal_code,latitude,longitude,stars,attributes,categories,review_count
business_id,,,,,,,,,,
f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,28031,35.462724,-80.852612,3.5,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...",36
Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",4
XNoUzKckATkOD1hP6vghZg,Felinus,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,NaN,"Pets, Pet Services, Pet Groomers",5
51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,85205,33.428065,-111.726648,4.5,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...",26
cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,Gilbert,AZ,85233,33.350399,-111.827142,4.5,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Automotive, Oil Change Stations, ...",38


In [11]:
df_r.head()

,business_id,review_stars,text,date
user_id,,,,
xDtS2iKsJuVUVzB2YhfPsg,f9NumwFMBDn751xgFiRbNA,4,driving half-hour get gun range getting old . ...,2012-06-26 00:48:30
46HhzhpBfTdTSB5ceTx_Og,f9NumwFMBDn751xgFiRbNA,4,"dad came area visit , wife mom went shopping d...",2012-05-29 23:44:04
vBxT4_bjFrbZEarWz6vsCQ,f9NumwFMBDn751xgFiRbNA,4,first time last time . 'm punk ! ! ! 22 9 mag ...,2017-06-18 00:24:54
qiz4Ri_cMSsoH3ccN19eAw,f9NumwFMBDn751xgFiRbNA,5,"besides great range , cheap . would cost cool ...",2016-02-14 02:28:57
0awwOxKGZVDt52vxkEX-eA,f9NumwFMBDn751xgFiRbNA,1,"place nice look , would n't really want stick ...",2014-05-28 20:00:46


In [12]:
df_c.head()

,total,dates
business_id,,
--1UhMGODdWsrMastO9DZw,7,"[datetime.datetime(2016, 4, 26, 19, 49, 16), d..."
--6MefnULPED_I942VcFNA,189,"[datetime.datetime(2011, 6, 4, 18, 22, 23), da..."
--7zmmkVg-IMGaXbuVd0SQ,193,"[datetime.datetime(2014, 12, 29, 19, 25, 50), ..."
--8LPVSo5i0Oo61X01sV9A,1,"[datetime.datetime(2016, 7, 8, 16, 43, 30)]"
--9QQLMTbFzLJ_oT-ON3Xw,38,"[datetime.datetime(2010, 6, 26, 17, 39, 7), da..."


In [13]:
df_t.head()

,user_id,text,date
business_id,,,
UYX5zL_Xj9WEc_Wp-FrqHw,hf27xTME3EiCp6NL6VtWZQ,Here for a quick mtg,2013-11-26 18:20:08
UYX5zL_Xj9WEc_Wp-FrqHw,Wi0VgIrbb8vqU6weyVw6tg,Surprised by the inventory! Was looking for a ...,2011-07-25 02:03:06
UYX5zL_Xj9WEc_Wp-FrqHw,5J4uykvpVCZ3pwceIKf45g,Thomas train table is a lot of fun!,2012-04-15 16:15:12
UYX5zL_Xj9WEc_Wp-FrqHw,dWkaK0k-5WSY4BJny1BtHw,Scanning books I want for Lydia and adding the...,2011-10-13 21:35:53
UYX5zL_Xj9WEc_Wp-FrqHw,8HXpvdxGR_yBQuA_T23Cxw,Mockingjay!!,2012-04-11 03:33:53


In [14]:
df_u.head()

,friends
user_id,
ntlvfPzc8eglqvk92iDIAw,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg..."
FOBRPlBHa3WPHFB5qYDlVg,"ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOug..."
zZUnPeh2hEp0WydbAZEOOg,"Uwlk0txjQBPw_JhHsQnyeg, Ybxr1tSCkv3lYA0I1qmnPQ..."
QaELAmRcDc5TfJEylaaP8g,"iog3Nyg1i4jeumiTVG_BSA, M92xWY2Vr9w0xoH8bPplfQ..."
xvu8G900tezTzbbfqmTKvA,"3W3ZMSthojCUirKEqAwGNw, eTIbuu23j9tOgmIa9POyLQ..."


In [12]:
### RUN ONLY ONCE ###

#######pre_processing.py########
# stop_words = set(stopwords.words('english'))
# def remove_stop_words(sentence):
#     if type(sentence)==str:
#         sentence = sentence.lower()
#         word_tokens = word_tokenize(sentence)
#         filtered_sentence = [w for w in word_tokens if not w in stop_words]   
#         merged_sentence = " ".join(filtered_sentence)
#         return merged_sentence
#     else:
#         return np.nan

# df_r['text'] = df_r.apply(lambda row : remove_stop_words(row['text']), axis = 1) 
# updated_dataset = df_r.to_csv('reviews_cleared.csv')
    

### Considering to keep the text of the review so that i can find similar reviews with cosine similarity
### It maybe better to filter data first with category so to make choices a lot of fewer

In [13]:
#### Deal only with data that belong to a specific category ####

def  categorize(top=10,category=False):
    df_explode = df_b.assign(categories = df_b.categories.str.split(', ')).explode('categories')["categories"]
    print("There are the top {} categories".format(top))
    print(df_explode.value_counts(normalize=True).nlargest(top))
    if category:
        a = input("Please specify category string")
        print("{} selected".format(a))
        df_b_specific = df_b[df_b['categories'].str.contains(a, na=False)]
    else:
        for val, cnt in df_explode.value_counts(normalize=True).nlargest(1).iteritems():
            print("{} selected".format(val))
            df_b_specific = df_b[df_b['categories'].str.contains(val, na=False)]
    df_r_specific = df_r[df_r['business_id'].isin(df_b_specific.index)]
    return df_b_specific,df_r_specific

In [14]:
bus,b = categorize()
b.shape

There are the top 10 categories
Restaurants                  0.061515
Shopping                     0.039849
Food                         0.034761
Home Services                0.028897
Health & Medical             0.024662
Beauty & Spas                0.024196
Local Services               0.020035
Automotive                   0.018398
Nightlife                    0.013737
Event Planning & Services    0.013292
Name: categories, dtype: float64
Restaurants selected


(4222976, 4)

In [101]:
scores = calculate_cosine(b,"greek greece feta souvlaki gyros tzatziki athens ")

In [102]:
sorted_scores_1 = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)

In [103]:
for key in  list(scores.keys()):
    star = RatingExtractor.get_rating_weight(bus.loc[[key]].stars.values[0])
    scores[key] = calculate_final_score(a[key],star)
sorted_scores_2 = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)

('w12EoDqaRXzN9KsouzRMyw', 0.008123911416994659)

In [86]:
def calculate_final_score(cs, r):
    amount = (cs / 100) * r
    return cs + amount

In [100]:
## Create cosine similarity for user reviews
## return only bussines_ids which reviews have cosine_similarity for specific keywords possitive
###############################################################################################
###                                  EXAMPLE                                                ###
### sorted_scores = calculate_cosine(b,"greek greece feta souvlaki gyros tzatziki athens ") ###
##############################################################################################

def calculate_cosine(b,sec):
    score_dict = {}
    for index, row in b.iterrows():
        try:
            a = CosineSimilarity.cosine_similarity_of(row['text'], sec)
            if a>0:
                if row['business_id'] not in score_dict:
                    score_dict[row['business_id']] = a
                else:
                    score_dict[row['business_id']] = (score_dict[row['business_id']]+a)/2.0
        except:
        ### There is one text with nan that causes exception ###    
            continue
    #sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)
    return score_dict

In [15]:
####### Cosine Similarity #######
import re, math
from collections import Counter

class CosineSimilarity:
    def __init__(self):
        print("Cosine Similarity initialized")

    @staticmethod
    def cosine_similarity_of(text1, text2):
        #get words first
        first = re.compile(r"[\w']+").findall(text1)
        second = re.compile(r"[\w']+").findall(text2)

        #get dictionary with each word and count.
        vector1 = Counter(first)
        vector2 = Counter(second)

        #convert vectors to set to find common words as intersection
        common = set(vector1.keys()).intersection(set(vector2.keys()))

        dot_product = 0.0

        for i in common:
            #get amount of each common word for both vectors and multiply them then add them together.
            dot_product += vector1[i] * vector2[i]

        squared_sum_vector1 = 0.0
        squared_sum_vector2 = 0.0

        #get squared sum values of word counts from each vector.
        for i in vector1.keys():
            squared_sum_vector1 += vector1[i]**2

        for i in vector2.keys():
            squared_sum_vector2 += vector2[i]**2

        #calculate magnitude with squared sums.
        magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

        if not magnitude:
           return 0.0
        else:
           return float(dot_product) / magnitude

In [16]:
from math import e
class RatingExtractor:
    def __init__(self):
        print("rating initialized")

    #Returns value between -q and q. for rating input between 0 and 10.
    #Parameters:
        #rating: indicates the rating for the destination
        #q: indicates the percentage of rating for general score. (default is 10.)
    @staticmethod
    def get_rating_weight(rating, q=10):
        if rating > 5 or rating < 0:
            return None
        else:
            m = (2*q) / 5 #10 because rating varies between 0 and 10
            b = -q
            return (m*rating) + b

In [17]:
##### Recommender Engine #####

import pandas as pd
from cosine_similarity import CosineSimilarity
import operator
import json

class RecommenderEngine:
    def __init__(self):
        print("engine initialized")
        
    def calculate_final_score(cs, r):
        amount = (cs / 100) * r
        return cs + amount

    def get_recommendations(keywords):

        df = pd.read_csv('reviews_cleared.csv')

        score_dict = {}

        for index, row in df.iterrows():
            score_dict[index] = CosineSimilarity.cosine_similarity_of(row['text'], keywords)

        #sort cities by score and index.
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        #create an empty results data frame.
        resultDF = pd.DataFrame(columns=('business_id', 'score'))

        #get highest scored 5 cities.
        for i in sorted_scores:
            #print index and score of the city.
            #print(i[0], i[1])
            resultDF = resultDF.append({'business_id': df.iloc[i[0]]['business_id'], 'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>4:
                break

        #convert DF to json.
        json_result = json.dumps(resultDF.to_dict('records'))
        return json_result

In [18]:
### Request ###

#from recommender_engine import RecommenderEngine
import json

culture_keywords = "history historical art architecture city culture"
beach_n_sun_keywords = "beach beaches park nature holiday sea seaside sand sunshine sun sunny"
nightlife_keywords = "nightclub nightclubs nightlife bar bars pub pubs party beer"

def get_recommendations(keywords):
    result = RecommenderEngine.get_recommendations(keywords)
    return result

def get_top_5_city_names_out_of_json(json_string):
    list = json.loads(json_string)
    result = []
    max = len(list)
    i = 0
    while i < max:
        result.append((list[i]['id'], list[i]['score']))
        i += 1

    return result

top_5_cultural_cities = get_recommendations(culture_keywords)
city_names_for_cultural = get_top_5_city_names_out_of_json(top_5_cultural_cities)
print(city_names_for_cultural)
print("#################")
top_5_summer_cities = get_recommendations(beach_n_sun_keywords)
city_names_for_summer = get_top_5_city_names_out_of_json(top_5_summer_cities)
print(city_names_for_summer)
print("#################")
top_5_party_cities = get_recommendations(nightlife_keywords)
city_names_for_party = get_top_5_city_names_out_of_json(top_5_party_cities)
print(city_names_for_party)
print("#################")

KeyboardInterrupt: 

In [47]:
df_c['test'] = df_c.apply(lambda row : to_date(row['date']), axis = 1)

In [45]:
def to_date(str_date):
    result = list()
    str_result = str_date.split(",")
    for i in str_result:
        if i[0] ==" ":
            result.append(datetime.datetime.strptime(i[1:], '%Y-%m-%d %H:%M:%S'))
        else:
            result.append(datetime.datetime.strptime(i, '%Y-%m-%d %H:%M:%S'))
    return result

In [ ]:
## Split categories names and count each category

df_explode = df_b.assign(categories = df_b.categories.str.split(', ')).explode('categories')["'categories'"]
#df_explode.categories.value_counts()
## keep only Restarants category
df_Restaurants = df_explode[df_explode["categories"]=="Restaurants"]

## keep only the reviews containig business_id in specific categories
df_r = df_r[df_r['business_id'].isin(df_Restaurants.index)]

import datetime

### Keep only scecific dates ###
date_time_str = '2019-01-01'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')
df_r = df_r[df_r['date']>= date_time_obj]
df_r = df_r[df_r['review_stars']>= 4]

In [ ]:
# Remove stop words and use TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english',max_features=1000)
# Construct TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(df_r[~(df_r['text'].isnull())]['text'])
tfidf_matrix.shape

In [ ]:
# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
# Get corresponding indices of the movies
#indices = pd.Series(df_b.index, index=df_b['business_id']).drop_duplicates()

##### Due to categories, mabe there is no need of similarity there, as i can just choose same category. I could use similarity to review text!

In [ ]:
# Recommendation function !!! need changes!!
def recommend(title, cosine_sim=cosine_sim):

    # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwise similarity scores of all movies with the given movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 15 most similar movies
    sim_scores = sim_scores[1:16]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Remove low-rated movies or outliers
    for i in movie_indices:
        pop = meta.at[i,'vote_average']
        if pop<5 or pop>10:
            movie_indices.remove(i)

    # Return the most similar movies qualifying the 5.0 rating threshold
    return meta[['original_title','vote_average']].iloc[movie_indices]

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate

In [ ]:
reader = Reader() # Used to parse a file containing ratings
df2 = Dataset.load_from_df(df[['user_id', 'business_id', 'review_stars']], reader)
kf = KFold(n_splits=5)
kf.split(df2) # Split the data into folds

In [ ]:
# Use Single Value Decomposition (SVD) for cross-validation and fitting
svd = SVD()
cross_validate(svd, df2, measures=['RMSE', 'MAE'])

trainset = df2.build_full_trainset()
svd.fit(trainset)

# TO DO
### use check-in to find the most visited places
### Use catogories to break data !!!!(Done with categorize func)
### Remove bussineses with categories out of interest for example guns
### Need to find a way to take advantage of review text (sure the is need of Vectorizer)
### I could find corellation between categories: for example food vs restaurant
### Efficient way for cosine similarity, one way is to use some keywords and then return only results with cosine similarity >0. In this way i will get all the bussiness_ids coresponding to my keywords)
